# The Tait Graph Algorithm

##### Here we present an implementation of the algorithm described in [] to compute the Tait graph from a PD-code. 

###### TODO: 1) Implement the directed edge from the silver-williams paper.
######       2) Implement handling of R1 and self loops
######       3) Implement finding the white graph when interior R1 moves are present

In [7]:
def color_and_plot_graph(g):
    red_edges = [e for e in g.edge_iterator() if e[2] == 1]
    g.plot(edge_color='blue', edge_colors={'red': red_edges}).show(figsize = 6)

In [2]:
import sage.graphs.graph as graph

def tait_graph(K):
    regions = [[abs(element) for element in sublist] for sublist in K.regions()]
    crossings = [set(pd) for pd in K.pd_code()]

    faces_static = {index: tuple(region) for index, region in enumerate(regions)}
    faces = {tuple(region): index for index, region in enumerate(regions)}
    edges = []
    seen = []

    for ind in range(len(faces)):
        common_list = [c for c in crossings if len(set(faces_static[ind]).intersection(c)) == 2]

        set_diff = [set(c).difference(faces_static[ind]) for c in common_list]

        neighboring_regions = [(key, value) for key, value in faces_static.items() for s in set_diff if len(set(value).intersection(s)) == 2]
        neighboring_regions_list = [region for region in regions for s in set_diff if len(set(region).intersection(s)) == 2]

        for i in range(len(neighboring_regions)):
            if neighboring_regions[i][1] not in seen:
                edges.append((faces[faces_static[ind]], neighboring_regions[i][0], weights(K.pd_code(), list(faces_static[ind]), neighboring_regions_list[i])))
        seen.append(faces_static[ind])
    G = graph.Graph(edges, multiedges=True, loops = True)
    components = G.connected_components()
    if len(components) > 2:
        raise ValueError('The link diagram is split.')
    return G.subgraph(components[0]), G.subgraph(components[1])

In [12]:
#find pd_code such that the union of region and neighboring region intersect with pd_code is exactly of length 4
def find_pd_code(pd_code, region, neighboring_region):
    print(f"region: {region}")
    combined = set(region).union(set(neighboring_region))
    for pd in pd_code:
        if len(combined.intersection(set(pd))) == 4:
            print(f"find_pd_code: {pd}")
            return pd

def matches_cyclic_order(corner, crossing):
    n = len(crossing)
    for i in range(n):
        if crossing[i] == corner[0] and crossing[(i + 1) % n] == corner[1]:
            return True
    return False

def weights(pd_code, region, neighboring_region):
    #the set of cyclic pairs for a region. can think of it as the corners of the crossings as one walks counter clockwise around the region
    #turning left at each crossing
    cyclic_pairs = [[region[i], region[(i + 1) % len(region)]] for i in range(len(region))]
    
    #TODO: implement differentiation between crossings in the find_pd_code between two regions that share more than one 'edge' (compatibility iwth R2 moves)

    crossing = find_pd_code(pd_code, region, neighboring_region)

    corner = [c for c in cyclic_pairs if len(set(c).intersection(set(crossing))) == 2]
    print(f"corner: {corner}")
    
    if len(corner) > 1:
        # Find the corner that matches the order in the crossing list
        corner = [corner for corner in corner if matches_cyclic_order(corner, crossing)]

    """
    check the index of the first value in the cyclic pairs list for each cyclic pair and see which strand it is in the pd_code
    the first element in a cyclic pair is the 'incoming' strand that forms the region
    if the corresponding index is 0 or 2, then it is the understrand, and thus the overstrand passes from left to right
    and the crossing is positive. if the index is 1 or 3, then it is a component of the overstrand, and the overstrand
    passes right to left and the crossing is negative
    """

    index_of_crossing = crossing.index(corner[0][0])    
    if index_of_crossing == 0 or index_of_crossing == 2:
        return 1
    else:
        return -1

In [5]:
def weighted_laplacian(G):
    edge_list = G.edges()
    vertices = G.get_vertices()
    vert_list = G.vertices()
    vert_dictionary = {value: key for key, value in enumerate(vertices)}
    print(vert_dictionary)
    n = G.order()
    M = Matrix(QQ, n, n)
    for e in edge_list:
        M[vert_dictionary[e[0]],vert_dictionary[e[1]]] += e[2]
        M[vert_dictionary[e[1]],vert_dictionary[e[0]]] += e[2]
        M[vert_dictionary[e[0]],vert_dictionary[e[0]]] -= e[2]
        M[vert_dictionary[e[1]],vert_dictionary[e[1]]] -= e[2]
    return M

def reduced_weighted_laplacian(M, i):
    return M.delete_rows([i]).delete_columns([i])

{0: 0, 2: 1, 5: 2, 8: 3, 9: 4}
21


### Chase's Type I/II Stuff

In [4]:
def abs_regions(self):
    regions = self.regions()
    return [set(abs(element) for element in reg) for reg in regions]

def weight(tuple, kt):
    if kt == 'TypeI':
        if int(tuple[2]) % 2 == 1:
            return +1
        else:
            return -1
    else:
        if int(tuple[2]) % 2 == 1:
            return -1
        else:
            return +1

#returns the length of the intersection of two sets. used primarily for the weights of the crossings, as a set is unordered.
def common_elements(lst, s):
    list_set = set(lst)
    common = list_set.intersection(s)
    return len(common)